In [2]:
import string
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score

def strip_punct(s):
    sentences = []
    exclude = set(string.punctuation)
    exclude.add(']')
    exclude.add('[')
    exclude.add('\xad')
    exclude.add('\n')
    exclude.add('\n1')
    exclude.add('\n2')
    exclude.add('\n3')
    exclude.add('•')
    exclude.add('«')
    exclude.add('»')
    for i in range(len(s)):
        sentences.append(''.join(ch for ch in str(s[i]).lower() if ch not in exclude))
    return sentences

def vector_prep(l):
    l1 = []
    l2 = []

    for i in range(len(l)):
        l_str = ''
        l1.append(re.split(r'[/&]', str(l[i])))
        for x in range (len(l1[i])):
            l_str += l1[i][x] + ' '
        l2.append(' '.join(l_str.strip().split()))
    return l2

def feature(l1, l2, l3):
    feature = []
    for i in range(len(l1)):
        feature.append(l1[i].lower() + ' ' + l2[i].lower() + ' ' + l3[i].lower())
    return feature

def cros_val(model, x_train_array, price_train, n_run):
    print ('Cross-validation: {}'.format(cross_val_score(model, x_train_array, price_train, cv=n_run)))

In [3]:
#1482535 rows x 8 columns
df = pd.read_csv('data/train.tsv', sep='\t')
df = df.sample(frac=1).reset_index(drop=True)

category_name_train = df['category_name'].iloc[14826:1482535].tolist()
name_train = df['name'].iloc[14826:1482535].tolist()
brand_train = df['brand_name'].iloc[14826:1482535].tolist()
price_train = df['price'].iloc[14826:1482535].tolist()

category_name_test = df['category_name'].iloc[0:14825].tolist()
name_test = df['name'].iloc[0:14825].tolist()
brand_test = df['brand_name'].iloc[0:14825].tolist()
price_test = df['price'].iloc[0:14825].tolist()

In [3]:
category_l_train = vector_prep(category_name_train)
name_l_train = vector_prep(name_train)
brand_l_train = vector_prep(brand_train)

category_l_test = vector_prep(category_name_test)
name_l_test = vector_prep(name_test)
brand_l_test = vector_prep(brand_test)

feature_train = feature(category_l_train, name_l_train, brand_l_train)
feature_test = feature(category_l_test, name_l_test, brand_l_test)

In [4]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(feature_train)
#x_train_array = X.toarray()

In [7]:
model = LinearRegression()
model_l = model.fit(X, price_train)

In [8]:
#corpus_test = ['small cat harness and leash Other Pet Supplies Others']
newVec = CountVectorizer(vocabulary=vectorizer.vocabulary_)
X_test = newVec.fit_transform(feature_test)
prices_predicted = model_l.predict(X_test).tolist()

In [9]:
n_run = 5 #number of runs
cros_val(model_l, X, price_train, n_run)

Cross-validation: [ 0.38858986  0.40132084  0.39405723  0.38794164  0.37951911]


In [10]:
for i in range(100):
    print (price_test[i], prices_predicted[i])

10.0 12.28601982610922
99.0 64.20870315482921
14.0 26.396400955423047
31.0 19.87622766069351
10.0 18.694033108617912
10.0 14.177766980207718
10.0 -14.67930119632523
50.0 40.45547452265314
8.0 18.464385018294642
10.0 14.664327432645285
28.0 25.551276763145054
8.0 0.885483098767768
20.0 23.713586231479027
115.0 49.26268433001228
7.0 0.11320681439170244
30.0 25.188863797877886
15.0 12.753560499879278
28.0 26.07305435837521
26.0 23.66639373222716
22.0 24.749277930624615
15.0 33.66389910054818
9.0 18.447411831928456
125.0 90.6414428314835
31.0 38.6074288094123
26.0 62.39439869651858
19.0 29.7983348907506
14.0 22.66880642944505
25.0 58.39567539328435
11.0 19.543545532899962
70.0 37.71096696790573
6.0 21.917633990401367
34.0 40.13882050304233
29.0 25.26643860201601
18.0 14.54273521957126
19.0 40.64665945668773
28.0 12.54484700848225
30.0 41.83286327756144
23.0 14.569084521922886
9.0 2.769089464552067
36.0 22.36106903594888
26.0 22.899785658973208
39.0 52.49439478291272
8.0 17.002903541918208
